# Facing the sad reality of python concurrency

Programming languages begin life in different ways, with different goals in mind when their inventors started building them. Unfortunately, concurrency isn't one of the goals when python started (unlike, say, Erlang and Elixir). As a result, concurrency do feel a bit foreign, cumbersome and unnatural in python. This is just something we have to live with.

In particular, when you start your python interactive interpreter, or when your python script starts to run, your code is in a single-threaded process where statements are executed one by one. What does "process" and "thread" mean here?

"Process" here means something different from the "process" in CSP: here it refers to an instance of your program that is executed for you by the operating system. A process has its own memory space and file descriptors provided by the operating system, and these are by default isolated from the other processes on the same system. A process may have one or more *threads of execution* running at the same time, with each thread executing its part of the code in sequence, but different threads can share memory (and potentially step onto each other, as we have previously seen), and due to interleaving of execution, the end result is often non-deterministic (or what we have previously refered to as "non-sequential").

Python supports threads by providing the `thread` and `threading` module, together with various locking primitives for concurrency control. However, even with all the locking stuff in place, threads in python has greatly diminished value compared to other languages. The reason is that the default python implementation has something called the global interpreter lock, or GIL, which roughly says that within each python process, at most one statement can be executed by the python runtime. Now this immediately excludes any possibility of utilising multiple process cores *within python itself* (as we will see later, if you step out of python there are ways around it). And locking has overheads. And python schedules thread execution in a somewhat unintuitive manner which results in it often favouring slow (or CPU-intensive) operations over fast ones, which is opposite of what most operating system does. The end result is that python code utilizing threads often runs *slower* than those not using threads. This picture isn't very encouraging.

Python also supports spawning processes within python itself using the `multiprocessing` module. However this module isn't that useful either: remember that by default processes don't share memory or resources, so inter-process communicating is restricted and cumbersome. And the overhead of using threads is even greater, even though the GIL restriction no longer applies because now we are in different processes.